In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.io.input import loadMidiFile
from src.io.output import pianoRoll, playPrettyMidi
from src import analysis
from src.adaptation import Adaptation
from src.datatypes.melody_data import MelodyData
from src.evaluation import Evaluation
from src.db.reference_sets import get_normalization_values_of_ref_set
from definitions import SequenceType

pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Instantiate Adaptation Object


In [2]:
a = Adaptation()

## Print available adaptation operations

In [3]:
for op in a.available_operations:
    print(op.__name__, ":", op.__doc__)

StartAndEndOnCTOperation : Creates a chord based on the estimated key of the control sequence and transposes the first and last note of the base sequence so they are chord tones (e.g. 'c', 'e' or 'g' for a C major chord).
TransposeNotesOperation : Estimates the key of both sequences and transposes every single note of the base sequence separately to the closest pitch that fits the key of the control sequence.
TransposeSequenceOperation : Estimates the key of both sequences and transposes the base sequence to be in the same key as the control sequence.


## Construct a pipeline

In [4]:
a.construct_pipeline(['StartAndEndOnCTOperation', 'TransposeNotesOperation'])
a.pipeline.get_operations()

['StartAndEndOnCTOperation', 'TransposeNotesOperation']

## Load 2 Melodies to emulate input and gen_base
+ print the notes of melody 2 (the base for adaptation)

In [5]:
midi1 = loadMidiFile('../midi/examples/monophonic/bohemian_mama2_4b.mid')
midi2 = loadMidiFile('../midi/examples/monophonic/mii1_4b.mid')

mel1 = MelodyData(midi1, SequenceType.REC_INPUT, {})
mel2 = MelodyData(midi2, SequenceType.GEN_BASE, {})

print("length:", len(mel2.sequence.instruments[0].notes))
mel2.sequence.instruments[0].notes

length: 18


[Note(start=0.005208, end=0.214583, pitch=66, velocity=73),
 Note(start=0.514583, end=0.610417, pitch=69, velocity=69),
 Note(start=0.761458, end=0.859375, pitch=73, velocity=68),
 Note(start=1.259375, end=1.355208, pitch=69, velocity=68),
 Note(start=1.752083, end=1.850000, pitch=66, velocity=68),
 Note(start=2.002083, end=2.116667, pitch=62, velocity=67),
 Note(start=2.266667, end=2.364583, pitch=62, velocity=62),
 Note(start=2.514583, end=2.610417, pitch=62, velocity=63),
 Note(start=3.752083, end=4.029167, pitch=61, velocity=62),
 Note(start=4.002083, end=4.116667, pitch=62, velocity=57),
 Note(start=4.266667, end=4.364583, pitch=66, velocity=68),
 Note(start=4.514583, end=4.610417, pitch=69, velocity=69),
 Note(start=4.761458, end=4.859375, pitch=73, velocity=68),
 Note(start=5.259375, end=5.355208, pitch=69, velocity=68),
 Note(start=5.752083, end=5.850000, pitch=66, velocity=68),
 Note(start=6.002083, end=6.760417, pitch=76, velocity=73),
 Note(start=6.761458, end=7.009375, pitc

## Adapt the melody and print the results

In [6]:
result, control = a.adapt(mel2, mel1)
print("length:", len(result.sequence.instruments[0].notes))
result.sequence.instruments[0].notes

META {'steps': [{'name': 'StartAndEndOnCTOperation', 'duration': 0.01599597930908203, 'intermediate_result': <music21.stream.Score 0x210bdfa23c8>}, {'name': 'TransposeNotesOperation', 'duration': 0.00599980354309082, 'intermediate_result': <music21.stream.Score 0x210bdfa23c8>}], 'total_duration': 0.02199578285217285}
length: 18


[Note(start=0.005208, end=0.214583, pitch=63, velocity=73),
 Note(start=0.514583, end=0.610417, pitch=69, velocity=69),
 Note(start=0.761458, end=0.859375, pitch=72, velocity=68),
 Note(start=1.259375, end=1.355208, pitch=69, velocity=68),
 Note(start=1.752083, end=1.850000, pitch=65, velocity=68),
 Note(start=2.002083, end=2.116667, pitch=62, velocity=67),
 Note(start=2.266667, end=2.364583, pitch=62, velocity=62),
 Note(start=2.514583, end=2.610417, pitch=62, velocity=63),
 Note(start=3.752083, end=4.029167, pitch=60, velocity=62),
 Note(start=4.002083, end=4.116667, pitch=62, velocity=57),
 Note(start=4.266667, end=4.364583, pitch=65, velocity=68),
 Note(start=4.514583, end=4.610417, pitch=69, velocity=69),
 Note(start=4.761458, end=4.859375, pitch=72, velocity=68),
 Note(start=5.259375, end=5.355208, pitch=69, velocity=68),
 Note(start=5.752083, end=5.850000, pitch=65, velocity=68),
 Note(start=6.002083, end=6.760417, pitch=75, velocity=73),
 Note(start=6.761458, end=7.009375, pitc

In [7]:
result.analysis

{'pitch_span': (<music21.pitch.Pitch C4>, <music21.pitch.Pitch E-5>)}

In [8]:
control.analysis

{'key': <music21.key.Key of g minor>}

# Evaluation

In [9]:
normalization_values = get_normalization_values_of_ref_set(1)
normalization_values

{'avg_ioi': 0.040211928,
 'avg_pitch_interval': 0.4,
 'note_count': 2,
 'note_length_histogram': 0.2018644384,
 'note_length_transition_matrix': 4.3588989435,
 'pitch_class_histogram': 0.2924397324,
 'pitch_class_histogram_per_bar': 0.2924397324,
 'pitch_class_transition_matrix': 4.2426406871,
 'pitch_count': 1,
 'pitch_range': 2,
 'avg_pitch_interval_distance': nan}

In [10]:
eval = Evaluation(normalization_values)
evaluation = eval.evaluate_similarity(result.sequence, control.sequence)
evaluation

{'absolute': {'pitch_count': 1.0,
  'pitch_class_histogram': 0.5147094163634272,
  'pitch_class_transition_matrix': 4.123105625617661,
  'avg_pitch_interval': 1.7058823529411764,
  'pitch_range': 5.0,
  'note_count': 0.0,
  'note_length_histogram': 0.4857326485917552,
  'note_length_transition_matrix': 14.491376746189438,
  'avg_ioi': 0.13480319245098077},
 'normalized': {'pitch_count': 1.0,
  'pitch_class_histogram': 1.7600529590808338,
  'pitch_class_transition_matrix': 0.9718253158119676,
  'avg_pitch_interval': 4.264705882352941,
  'pitch_range': 2.5,
  'note_count': 0.0,
  'note_length_histogram': 2.406231887308761,
  'note_length_transition_matrix': 3.3245498310528654,
  'avg_ioi': 3.352318557095317}}